In [1]:
import bittensor as bt
from bittensor.core.extrinsics.pallets import SubtensorModule


wallet = bt.Wallet("auto_staker")


In [2]:
s = bt.Subtensor()

In [158]:
NETUID = 15
vali_hk = "5E2LP6EnZ54m3wS8s1yPvD5c3xo71kQroBw7aUVK32TKeZ5u"
stakes = s.get_stake(wallet.coldkey.ss58_address, vali_hk, NETUID)
print(stakes)

if NETUID == 47:
    entry_price = bt.Balance.from_float(0.0031730955)  # 47
elif NETUID == 99:
    entry_price = bt.Balance.from_float(0.003528)  # 99
elif NETUID == 15:
    entry_price = bt.Balance.from_float(0.003576)  # 15
entry_price

SLIPPAGE = 0.025
current_price = s.get_subnet_price(NETUID)
limit_price = bt.Balance.from_float(current_price.tao * (1 - SLIPPAGE))
current_price
print("current_price", current_price)
print("limit", limit_price)
print("profit", current_price.rao / entry_price.rao * (1 - SLIPPAGE))

‎15.510080341ο‎
current_price τ0.003769349
limit τ0.003675115
profit 1.0277167994966443


In [159]:
import math

RAO_PER_TAO = 10**9

EXTRINSIC_FEE_TAO_REMOVE_STAKE = 0.000135688
ALPHA_FEE_PCT = 0.0005


def breakeven_limit_price_rao(
    *,
    entry_price_rao_per_alpha: int,
    amount_unstaked_rao: int,
    min_fill_pct: float = 1.0,  # e.g. 1.0 for full fill, 0.1 for 10% worst-case partial
    alpha_fee_pct: float = ALPHA_FEE_PCT,
    extrinsic_fee_tao: float = EXTRINSIC_FEE_TAO_REMOVE_STAKE,
) -> int:
    """Return break-even limit price (RAO per 1 ALPHA) including alpha fee + flat extrinsic fee."""
    # Executed (filled) alpha amount (in rao)
    filled_alpha_rao = int(amount_unstaked_rao * min_fill_pct)
    if filled_alpha_rao <= 0:
        raise ValueError(
            "filled_alpha_rao must be > 0. Increase min_fill_pct or amount_unstaked_rao."
        )

    # Alpha fee is paid in Alpha (deducted from the amount that gets swapped)
    swapped_alpha_rao = int(filled_alpha_rao * (1.0 - alpha_fee_pct))
    if swapped_alpha_rao <= 0:
        raise ValueError(
            "swapped_alpha_rao must be > 0. Check alpha_fee_pct and amounts."
        )

    extrinsic_fee_rao = int(extrinsic_fee_tao * RAO_PER_TAO)

    # Cost basis in TAO (rao): (alpha_rao * entry_price_rao_per_alpha) / 1e9
    cost_basis_tao_rao = (filled_alpha_rao * entry_price_rao_per_alpha) // RAO_PER_TAO

    # Need: (swapped_alpha_rao * limit_price_rao_per_alpha) / 1e9 - extrinsic_fee_rao >= cost_basis_tao_rao
    # => limit_price >= (cost_basis + extrinsic) * 1e9 / swapped_alpha_rao
    limit_price_rao_per_alpha = math.ceil(
        ((cost_basis_tao_rao + extrinsic_fee_rao) * RAO_PER_TAO) / swapped_alpha_rao
    )
    return int(limit_price_rao_per_alpha)


# --- usage with your variables ---
SLIPPAGE = 0.025
current_price = s.get_subnet_price(NETUID)  # has .rao and .tao
amount_unstaked_rao = stakes.rao

# Choose your "guarantee" level:
# - 1.0  -> guarantee only if full amount fills
# - 0.1  -> guarantee even if only 10% fills (more strict limit)
min_fill_pct = 1

profit_floor_limit_rao = breakeven_limit_price_rao(
    entry_price_rao_per_alpha=entry_price.rao,
    amount_unstaked_rao=amount_unstaked_rao,
    min_fill_pct=min_fill_pct,
)

# Optional: also enforce a slippage-based floor from current spot (less important if profit_floor is stricter)
slippage_limit_rao = int(current_price.rao * (1.0 - SLIPPAGE))

# Pick the stricter one (higher floor) -> safer, but more partial/fails
final_limit_rao = max(profit_floor_limit_rao, slippage_limit_rao)

limit_price = bt.Balance.from_rao(final_limit_rao)

print("current_price", current_price)
print("profit_floor_limit", bt.Balance.from_rao(profit_floor_limit_rao))
print("slippage_limit", bt.Balance.from_rao(slippage_limit_rao))
print("final_limit", limit_price)
print("final_pct", limit_price.rao / entry_price.rao)

current_price τ0.003769349
profit_floor_limit τ0.003586542
slippage_limit τ0.003675115
final_limit τ0.003675115
final_pct 1.0277167225950783


In [160]:
call = SubtensorModule(subtensor=s).remove_stake_limit(
    netuid=NETUID,
    hotkey=vali_hk,
    amount_unstaked=stakes.rao,
    limit_price=limit_price.rao,
    allow_partial=True,
)
call
extrinsic_data = {"call": call, "keypair": wallet.coldkey}
current_block = s.get_current_block()
print(current_block)
extrinsic_data["era"] = {"period": 4, "current": current_block - 2}
extrinsic = s.substrate.create_signed_extrinsic(**extrinsic_data)
bt.logging.debug(f"Prepared extrinsic: {extrinsic}")
response = s.substrate.submit_extrinsic(
    extrinsic=extrinsic,
    wait_for_inclusion=True,
    wait_for_finalization=False,
)
ok = response.is_success
print(
    f"Response: {response} | succes: {ok} | error: { response.error_message if not ok else None}"
)

7488273
Response: <async_substrate_interface.sync_substrate.ExtrinsicReceipt object at 0x7d54ec241f40> | succes: True | error: None


In [161]:
vali_hk = "5E2LP6EnZ54m3wS8s1yPvD5c3xo71kQroBw7aUVK32TKeZ5u"
stakes = s.get_stake(wallet.coldkey.ss58_address, vali_hk, NETUID)
stakes

‎0.000000000ο‎